In [1]:
# lets create a simple BPE tokenizer

In [68]:
with open('data.txt', 'r') as f:
    data = f.read()

In [69]:
data = data.replace("\n", " ").replace(
    "===", "").replace("==", "").replace("\\n", " ")

In [70]:
def get_stats(ids, counts=None):
    if counts is None:
        counts = {}
    for pair in zip(ids, ids[1:]):
        if pair not in counts:
            counts[pair] = 0
        counts[pair] += 1
    return counts 

In [71]:
def merge(ids, pair, idx):
    new_ids = []
    i = 0
    while i < len(ids):
        if ids[i] == pair[0] and i < len(ids) - 1 and ids[i + 1] == pair[1]:
            new_ids.append(idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids


In [74]:
# now put it all together into a tokenizer function
def train_tokenizer(data, num_merges):
    text_bytes = data.encode('utf-8')
    ids = list(text_bytes)

    merges = {} # (int, int) -> int
    vocab = {idx: bytes([idx]) for idx in range(256)} # int -> bytes 

    for i in range(num_merges):
        stats = get_stats(ids)
        pair = max(stats, key=stats.get)
        idx = 256 + i
        ids = merge(ids, pair, idx)
        merges[pair] = idx
        vocab[idx] = vocab[pair[0]] + vocab[pair[1]]

    return vocab, merges

In [73]:
def encode(text, merges):
    ids = list(text.encode('utf-8'))
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and (ids[i], ids[i + 1]) in merges:
            ids = merge(ids, (ids[i], ids[i + 1]), merges[(ids[i], ids[i + 1])])
        i += 1
    return ids

def decode(ids, vocab):
    text = b""
    for i in ids:
        text += vocab[i]
    return text.decode('utf-8')


In [ ]:
vocab, merges = train_tokenizer(data, 10)
encoded = encode(data, merges)
decoded = decode(encoded, vocab)

assert data == decoded